In [2]:
import win32com.client as win32
import openpyxl
import pandas as pd
import ctypes
import win32com
import time
import shutil
import warnings
import os
from pathlib import Path
from openpyxl.styles import Font, NamedStyle, Alignment


start_time = time.time()
chisl = str(Path.cwd() / f'chisl/Численность ВВБ.xlsx') #№1 таблица Численность ВВБ
name_s = os.path.basename(chisl)
sprVVB = str(Path.cwd() / f'sprVVB/Справочник.xlsx') #№2 таблица Справочник
name_s = os.path.basename(sprVVB)
sprBOSS = str(Path.cwd() / f'sprBOSS/Боссы.xlsx') #№3 таблица Справочник
name_s = os.path.basename(sprBOSS)
shabl = str(Path.cwd() / f'shabl/ВВБ_AntiCOV.xlsx') #№5 таблица (шаблон для формирования результата)
file_cov = str(Path.cwd() /f"file_cov/ВВБ_AntiCOV1.xlsx")  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

print('загрузка данных из Численности ВВБ')

#чтение таблицы 1 - Численность ВВБ
df1 = pd.read_excel(chisl, sheet_name='ВВБ',
                   usecols=["ТН", "ФИО", "Должность",  "Внутренний e-mail", "Внешний e-mail", "Табельный номер руководителя",
                            "ФИО руководителя", "Должность руководителя","Подразделение 5","Подразделение 6","Подразделение 7","Подразделение 8","Подразделение 11",
                            "Функциональный блок", "Область","Подразделение 2","Подразделение 3","Подразделение 4","Подразделение 9","Подразделение 10","Подразделение 12"])

df1.fillna("-", inplace=True) 

print('DF1 chisl - Done')
lst_t1 = df1.ТН.tolist()
print("список ТН - Done") 
df1 = df1.set_index('ТН') 

#чтение таблицы 2 - Справочник
df2 = pd.read_excel(sprVVB, sheet_name='Справочник',
                   usecols=["Наименование полное", "ЦА/ТБ/ПЦП", "ВСП/КИЦ","ВИП ВСП","Подразделение кратко","Принадлежность","Площадка ОСЦ", "Подразделение ОСЦ", "Блок 2", "Клиентские подразделения"], dtype=str)

df2.fillna("-", inplace=True) 
df2['Наименование полное'] = df2['Наименование полное'].astype(str)
df2 = df2.rename(columns={"Наименование полное": "НП"})

print('DF2 sprVVB - Done')
lst_t2 = df2.НП.tolist()
print("список НП - Done") 
df2 = df2.set_index('НП') 


#чтение таблицы 2 -Боссы
df3 = pd.read_excel(sprBOSS, sheet_name='Боссы')

df3.fillna("-", inplace=True) 
df3['ТН'] = df3['ТН'].astype(int)

print('DF3 sprBOSS - Done')
lst_t3 = df3.ТН.tolist()
print("список ТН - Done") 
df3 = df3.set_index('ТН') 


print('download shablon')
start_time_2 = time.time()  # время запуска
wb = openpyxl.load_workbook(shabl)

ws = wb.active
n = 0
sheets = wb.sheetnames
wsa = wb[sheets[n]]
# print(wsa)
rows = wsa.max_row 
cols = wsa.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa из Численности ВВБ

for i in range(2, rows+1):

    if wsa.cell(row=i, column=2).value in lst_t1:

        wsa.cell(row=i, column=1).value = df1.loc[wsa.cell(row=i, column=2).value, "ФИО"]
        wsa.cell(row=i, column=3).value = df1.loc[wsa.cell(row=i, column=2).value, "Должность"]
        wsa.cell(row=i, column=4).value = df1.loc[wsa.cell(row=i, column=2).value, "Внутренний e-mail"]
        wsa.cell(row=i, column=5).value = df1.loc[wsa.cell(row=i, column=2).value, "Внешний e-mail"]
        wsa.cell(row=i, column=6).value = df1.loc[wsa.cell(row=i, column=2).value, "Табельный номер руководителя"]
        wsa.cell(row=i, column=7).value = df1.loc[wsa.cell(row=i, column=2).value, "ФИО руководителя"]
        wsa.cell(row=i, column=8).value = df1.loc[wsa.cell(row=i, column=2).value, "Должность руководителя"]
        wsa.cell(row=i, column=9).value = df1.loc[wsa.cell(row=i, column=2).value, "Функциональный блок"]
        wsa.cell(row=i, column=10).value = df1.loc[wsa.cell(row=i, column=2).value, "Область"]
        wsa.cell(row=i, column=11).value = df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 2"]
        wsa.cell(row=i, column=12).value = df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 3"]
        a2=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 2"]
        a3=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 3"]
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 4"]!="-":
            a4=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 4"]
        else:a4=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 5"]!="-":
            a5=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 5"]
        else:a5=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 6"]!="-":
            a6=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 6"]
        else:a6=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 7"]!="-":
            a7=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 7"]
        else:a7=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 8"]!="-":
            a8=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 8"]
        else:a8=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 9"]!="-":
            a9=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 9"]
        else:a9=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 10"]!="-":
            a10=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 10"]
        else:a10=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 11"]!="-":
            a11=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 11"]
        else:a11=""
        if df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 12"]!="-":
            a12=df1.loc[wsa.cell(row=i, column=2).value, "Подразделение 12"]
        else:a12=""
        wsa.cell(row=i, column=13).value = a2+"/"+a3+"/"+a4+"/"+a5+"/"+a6+"/"+a7+"/"+a8+"/"+a9+"/"+a10+"/"+a11+"/"+a12
        
        if wsa.cell(row=i, column=13).value !='-' and  wsa.cell(row=i, column=13).value !='':        

            if wsa.cell(row=i, column=13).value in lst_t2:
                wsa.cell(row=i, column=14).value = df2.loc[wsa.cell(row=i, column=13).value, "ЦА/ТБ/ПЦП"]
                wsa.cell(row=i, column=15).value = df2.loc[wsa.cell(row=i, column=13).value, "ВСП/КИЦ"]
                wsa.cell(row=i, column=17).value = df2.loc[wsa.cell(row=i, column=13).value, "Подразделение кратко"]
                wsa.cell(row=i, column=18).value = df2.loc[wsa.cell(row=i, column=13).value, "Принадлежность"]
                wsa.cell(row=i, column=22).value = df2.loc[wsa.cell(row=i, column=13).value, "Блок 2"]
                wsa.cell(row=i, column=61).value = df2.loc[wsa.cell(row=i, column=13).value, "Клиентские подразделения"]
                
                if wsa.cell(row=i, column=61).value=='nan':
                    wsa.cell(row=i, column=61).value=""

            
            if wsa.cell(row=i, column=13).value in lst_t2:
                wsa.cell(row=i, column=16).value = df2.loc[wsa.cell(row=i, column=13).value, "ВИП ВСП"]
                if df2.loc[wsa.cell(row=i, column=13).value, "ВИП ВСП"]=='nan':
                    wsa.cell(row=i, column=16).value =""
            else: wsa.cell(row=i, column=16).value =""

           
            if wsa.cell(row=i, column=13).value in lst_t2:
                wsa.cell(row=i, column=19).value = df2.loc[wsa.cell(row=i, column=13).value, "Площадка ОСЦ"]
                if df2.loc[wsa.cell(row=i, column=13).value, "Площадка ОСЦ"]=='nan':
                    wsa.cell(row=i, column=19).value =""
            else: wsa.cell(row=i, column=19).value =""
                
            if wsa.cell(row=i, column=13).value in lst_t2:
                wsa.cell(row=i, column=20).value = df2.loc[wsa.cell(row=i, column=13).value, "Подразделение ОСЦ"]
                if df2.loc[wsa.cell(row=i, column=13).value, "Подразделение ОСЦ"]=='nan':
                    wsa.cell(row=i, column=20).value =""
            else: wsa.cell(row=i, column=20).value =""
            
                
            if wsa.cell(row=i, column=13).value!='':
                wsa.cell(row=i, column=21).value =1
                
        if wsa.cell(row=i, column=13).value !='-' and  wsa.cell(row=i, column=13).value !='':   
                
            if wsa.cell(row=i, column=2).value in lst_t3:
        
                wsa.cell(row=i, column=22).value = df3.loc[wsa.cell(row=i, column=2).value, "Блок 2"]

            
print('переменные')

wb.save(file_cov)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_cov}')

# wb = excel.Workbooks.Open(shabl

загрузка данных из Численности ВВБ
DF1 chisl - Done
список ТН - Done
DF2 sprVVB - Done
список НП - Done
DF3 sprBOSS - Done
список ТН - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\Справочник для COVID-19\file_cov\ВВБ_AntiCOV1.xlsx
